In [23]:
import requests
import pandas as pd
from datetime import datetime, timedelta

LOKI_URL = "http://localhost:3100/loki/api/v1/query_range"

# Dictionnaire des signatures d'attaques (Patterns de détection)
THREAT_SIGNATURES = {
    "SQL Injection": '{container="web_server"} | json | request_uri =~ ".*(%27|OR|1%3D1|--).*"',
    "Path Traversal": '{container="web_server"} | json | request_uri =~ ".*etc/passwd.*"',
    "Nmap Scan": '{container="web_server"} | json | http_user_agent =~ ".*Nmap.*"',
    "SSH Brute Force": '{container="victim_server"} |= "Failed password"'
}

def hunt_threats():
    end_time = datetime.now()
    start_time = end_time - timedelta(minutes=10)
    
    print(f"--- 🕵️‍♂️ SESSION DE THREAT HUNTING ({end_time.strftime('%H:%M')}) ---")
    
    for attack_type, query in THREAT_SIGNATURES.items():
        params = {
            'query': query,
            'start': int(start_time.timestamp() * 1e9),
            'limit': 500
        }
        
        try:
            r = requests.get(LOKI_URL, params=params)
            results = r.json()['data']['result']
            count = sum(len(res['values']) for res in results)
            
            status = "🔴 DÉTECTÉ" if count > 0 else "🟢 RAS"
            print(f"[{status}] {attack_type.ljust(20)} : {count} occurrences")
            
            # Si on détecte une SQLi, on affiche les IPs attaquantes
            if count > 0 and "SQL" in attack_type:
                for res in results:
                    ip = res['stream'].get('remote_addr', 'Inconnue')
                    print(f"   ↳ IP Source suspecte : {ip}")
                    
        except Exception as e:
            print(f"❌ Erreur sur {attack_type}: {e}")

hunt_threats()

--- 🕵️‍♂️ SESSION DE THREAT HUNTING (01:16) ---
[🔴 DÉTECTÉ] SQL Injection        : 30 occurrences
   ↳ IP Source suspecte : 172.28.0.4
   ↳ IP Source suspecte : 172.28.0.4
   ↳ IP Source suspecte : 172.28.0.4
[🔴 DÉTECTÉ] Path Traversal       : 30 occurrences
[🔴 DÉTECTÉ] Nmap Scan            : 3 occurrences
[🔴 DÉTECTÉ] SSH Brute Force      : 36 occurrences
